In [ ]:
# %pip install python-dotenv
# %pip install seaborn
# %pip install tensorflow_data_validation

In [ ]:
import os
import pandas as pd
import geopandas as gpd
import pygeos as pg
import numpy as np
import tensorflow as tf
import tensorflow_data_validation as tfdv
import sqlalchemy as sq
import sklearn as sk
import scipy as sp
import seaborn as sns
from datetime import datetime
from dotenv import load_dotenv
from IPython.display import clear_output
from matplotlib import pyplot as plt
from DataService import DataService

In [ ]:
# The following lines adjust the granularity of reporting.
#pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
os.chdir('/tf')
PGUSER = os.getenv('POSTGRES_USER')
PGPW = os.getenv('POSTGRES_PW')
PGDB = os.getenv('POSTGRES_DB')
NULLFLAG = -9999
ONEDAY = 24 * 60 * 60
WEEK = 7 * ONEDAY

In [ ]:
# Connect to the database
pullService = DataService(PGDB, PGUSER, PGPW)
db_pull_con = pullService.connect()

pushService = DataService(PGDB, PGUSER, PGPW)
db_push_con = pushService.connect()

In [ ]:
# load firewaterelev
fireWaterElev = "FireWaterElev"
query1 = "SELECT * FROM public.\"{}\";".format(fireWaterElev)
dfFireWaterElev = pd.read_sql(query1, db_pull_con)

In [ ]:
print(dfFireWaterElev.count())
print(dfFireWaterElev.describe())
print(dfFireWaterElev.isnull().sum().sum())

In [ ]:
# Load the data
fireWeather = "FireWeather"
query2 = "SELECT * FROM public.\"{}\";".format(fireWeather)
dfFireWeather = pd.read_sql(query2, db_pull_con)

In [ ]:
print(dfFireWeather.count())
print(dfFireWeather.describe())
print(dfFireWeather.isnull().sum().sum())

In [ ]:
# Load WeatherDataHourlyAggDaily
weatherDataHourlyAggDaily = "WeatherDataHourlyAggDaily"
query3 = "SELECT * FROM public.\"{}\";".format(weatherDataHourlyAggDaily)
dfWeatherDaily = pd.read_sql(query3, db_pull_con)

In [ ]:
print(dfWeatherDaily.count())
print(dfWeatherDaily.describe())
print(dfWeatherDaily.isnull().sum().sum())

Sums of max temp humidity days with precipitation in time period

In [ ]:
dfFWEW = dfFireWaterElev.copy(deep=True)

In [ ]:
# add 7 day sum max temp column, 7 day sum humidity column, 7 day sum precip column
for index, row in dfFWEW.iterrows():
    dfFWEW.at[index, '7daySumMaxTemp'] = dfWeatherDaily.loc[(dfWeatherDaily['ClimateID'] == row['ClimateID']) & (dfWeatherDaily['utc'] >= (row['utc'] - WEEK)) & (dfWeatherDaily['utc'] <= row['utc'])]['MaxTemp'].sum()
    dfFWEW.at[index, '7daySumHumidity'] = dfWeatherDaily.loc[(dfWeatherDaily['ClimateID'] == row['ClimateID']) & (dfWeatherDaily['utc'] >= (row['utc'] - WEEK)) & (dfWeatherDaily['utc'] <= row['utc'])]['MaxHumidity'].sum()
    dfFWEW.at[index, '7daySumPrecip'] = dfWeatherDaily.loc[(dfWeatherDaily['ClimateID'] == row['ClimateID']) & (dfWeatherDaily['utc'] >= (row['utc'] - WEEK)) & (dfWeatherDaily['utc'] <= row['utc'])]['TotalPrecip'].sum()

In [ ]:
print(dfFWEW.count())
print(dfFWEW.describe())
print(dfFWEW.isnull().sum().sum())

In [ ]:
# add 14 day sum max temp column, 14 day sum humidity column, 14 day sum precip column
for index, row in dfFWEW.iterrows():
    dfFWEW.at[index, '14daySumMaxTemp'] = dfWeatherDaily.loc[(dfWeatherDaily['ClimateID'] == row['ClimateID']) & (dfWeatherDaily['utc'] >= (row['utc'] - 2 * WEEK)) & (dfWeatherDaily['utc'] <= row['utc'])]['MaxTemp'].sum()
    dfFWEW.at[index, '14daySumHumidity'] = dfWeatherDaily.loc[(dfWeatherDaily['ClimateID'] == row['ClimateID']) & (dfWeatherDaily['utc'] >= (row['utc'] - 2 * WEEK)) & (dfWeatherDaily['utc'] <= row['utc'])]['MaxHumidity'].sum()
    dfFWEW.at[index, '14daySumPrecip'] = dfWeatherDaily.loc[(dfWeatherDaily['ClimateID'] == row['ClimateID']) & (dfWeatherDaily['utc'] >= (row['utc'] - 2 * WEEK)) & (dfWeatherDaily['utc'] <= row['utc'])]['TotalPrecip'].sum()

In [ ]:
# add 21 day sum max temp column, 21 day sum humidity column, 21 day sum precip column
for index, row in dfFWEW.iterrows():
    dfFWEW.at[index, '21daySumMaxTemp'] = dfWeatherDaily.loc[(dfWeatherDaily['ClimateID'] == row['ClimateID']) & (dfWeatherDaily['utc'] >= (row['utc'] - 3 * WEEK)) & (dfWeatherDaily['utc'] <= row['utc'])]['MaxTemp'].sum()
    dfFWEW.at[index, '21daySumHumidity'] = dfWeatherDaily.loc[(dfWeatherDaily['ClimateID'] == row['ClimateID']) & (dfWeatherDaily['utc'] >= (row['utc'] - 3 * WEEK)) & (dfWeatherDaily['utc'] <= row['utc'])]['MaxHumidity'].sum()
    dfFWEW.at[index, '21daySumPrecip'] = dfWeatherDaily.loc[(dfWeatherDaily['ClimateID'] == row['ClimateID']) & (dfWeatherDaily['utc'] >= (row['utc'] - 3 * WEEK)) & (dfWeatherDaily['utc'] <= row['utc'])]['TotalPrecip'].sum()

In [ ]:
# add 28 day sum max temp column, 28 day sum humidity column, 28 day sum precip column
for index, row in dfFWEW.iterrows():
    dfFWEW.at[index, '28daySumMaxTemp'] = dfWeatherDaily.loc[(dfWeatherDaily['ClimateID'] == row['ClimateID']) & (dfWeatherDaily['utc'] >= (row['utc'] - 4 * WEEK)) & (dfWeatherDaily['utc'] <= row['utc'])]['MaxTemp'].sum()
    dfFWEW.at[index, '28daySumHumidity'] = dfWeatherDaily.loc[(dfWeatherDaily['ClimateID'] == row['ClimateID']) & (dfWeatherDaily['utc'] >= (row['utc'] - 4 * WEEK)) & (dfWeatherDaily['utc'] <= row['utc'])]['MaxHumidity'].sum()
    dfFWEW.at[index, '28daySumPrecip'] = dfWeatherDaily.loc[(dfWeatherDaily['ClimateID'] == row['ClimateID']) & (dfWeatherDaily['utc'] >= (row['utc'] - 4 * WEEK)) & (dfWeatherDaily['utc'] <= row['utc'])]['TotalPrecip'].sum()

In [ ]:
# add 35 day sum max temp column, 35 day sum humidity column, 35 day sum precip column
for index, row in dfFWEW.iterrows():
    dfFWEW.at[index, '35daySumMaxTemp'] = dfWeatherDaily.loc[(dfWeatherDaily['ClimateID'] == row['ClimateID']) & (dfWeatherDaily['utc'] >= (row['utc'] - 5 * WEEK)) & (dfWeatherDaily['utc'] <= row['utc'])]['MaxTemp'].sum()
    dfFWEW.at[index, '35daySumHumidity'] = dfWeatherDaily.loc[(dfWeatherDaily['ClimateID'] == row['ClimateID']) & (dfWeatherDaily['utc'] >= (row['utc'] - 5 * WEEK)) & (dfWeatherDaily['utc'] <= row['utc'])]['MaxHumidity'].sum()
    dfFWEW.at[index, '35daySumPrecip'] = dfWeatherDaily.loc[(dfWeatherDaily['ClimateID'] == row['ClimateID']) & (dfWeatherDaily['utc'] >= (row['utc'] - 5 * WEEK)) & (dfWeatherDaily['utc'] <= row['utc'])]['TotalPrecip'].sum()

In [ ]:
# add 42 day sum max temp column, 42 day sum humidity column, 42 day sum precip column
for index, row in dfFWEW.iterrows():
    dfFWEW.at[index, '42daySumMaxTemp'] = dfWeatherDaily.loc[(dfWeatherDaily['ClimateID'] == row['ClimateID']) & (dfWeatherDaily['utc'] >= (row['utc'] - 6 * WEEK)) & (dfWeatherDaily['utc'] <= row['utc'])]['MaxTemp'].sum()
    dfFWEW.at[index, '42daySumHumidity'] = dfWeatherDaily.loc[(dfWeatherDaily['ClimateID'] == row['ClimateID']) & (dfWeatherDaily['utc'] >= (row['utc'] - 6 * WEEK)) & (dfWeatherDaily['utc'] <= row['utc'])]['MaxHumidity'].sum()
    dfFWEW.at[index, '42daySumPrecip'] = dfWeatherDaily.loc[(dfWeatherDaily['ClimateID'] == row['ClimateID']) & (dfWeatherDaily['utc'] >= (row['utc'] - 6 * WEEK)) & (dfWeatherDaily['utc'] <= row['utc'])]['TotalPrecip'].sum()

In [ ]:
# add 49 day sum max temp column, 49 day sum humidity column, 49 day sum precip column
for index, row in dfFWEW.iterrows():
    dfFWEW.at[index, '49daySumMaxTemp'] = dfWeatherDaily.loc[(dfWeatherDaily['ClimateID'] == row['ClimateID']) & (dfWeatherDaily['utc'] >= (row['utc'] - 7 * WEEK)) & (dfWeatherDaily['utc'] <= row['utc'])]['MaxTemp'].sum()
    dfFWEW.at[index, '49daySumHumidity'] = dfWeatherDaily.loc[(dfWeatherDaily['ClimateID'] == row['ClimateID']) & (dfWeatherDaily['utc'] >= (row['utc'] - 7 * WEEK)) & (dfWeatherDaily['utc'] <= row['utc'])]['MaxHumidity'].sum()
    dfFWEW.at[index, '49daySumPrecip'] = dfWeatherDaily.loc[(dfWeatherDaily['ClimateID'] == row['ClimateID']) & (dfWeatherDaily['utc'] >= (row['utc'] - 7 * WEEK)) & (dfWeatherDaily['utc'] <= row['utc'])]['TotalPrecip'].sum()

In [ ]:
# add 56 day sum max temp column, 56 day sum humidity column, 56 day sum precip column
for index, row in dfFWEW.iterrows():
    dfFWEW.at[index, '56daySumMaxTemp'] = dfWeatherDaily.loc[(dfWeatherDaily['ClimateID'] == row['ClimateID']) & (dfWeatherDaily['utc'] >= (row['utc'] - 8 * WEEK)) & (dfWeatherDaily['utc'] <= row['utc'])]['MaxTemp'].sum()
    dfFWEW.at[index, '56daySumHumidity'] = dfWeatherDaily.loc[(dfWeatherDaily['ClimateID'] == row['ClimateID']) & (dfWeatherDaily['utc'] >= (row['utc'] - 8 * WEEK)) & (dfWeatherDaily['utc'] <= row['utc'])]['MaxHumidity'].sum()
    dfFWEW.at[index, '56daySumPrecip'] = dfWeatherDaily.loc[(dfWeatherDaily['ClimateID'] == row['ClimateID']) & (dfWeatherDaily['utc'] >= (row['utc'] - 8 * WEEK)) & (dfWeatherDaily['utc'] <= row['utc'])]['TotalPrecip'].sum()

In [ ]:
print(dfFWEW.count())
print(dfFWEW.describe())
print(dfFWEW.isnull().sum().sum())

In [ ]:
# group the fires with same utc and average the N day columns
dfFWEW = dfFWEW.groupby(['utc', 'YEAR', 'MONTH', 'DAY'], as_index=False).agg({'SIZE_HA':['sum'],
                                                     '7daySumMaxTemp':['mean'], '7daySumHumidity':['mean'], '7daySumPrecip':['mean'],
                                                     '14daySumMaxTemp':['mean'], '14daySumHumidity':['mean'], '14daySumPrecip':['mean'],
                                                     '21daySumMaxTemp':['mean'], '21daySumHumidity':['mean'], '21daySumPrecip':['mean'],
                                                     '28daySumMaxTemp':['mean'], '28daySumHumidity':['mean'], '28daySumPrecip':['mean'],
                                                     '35daySumMaxTemp':['mean'], '35daySumHumidity':['mean'], '35daySumPrecip':['mean'],
                                                     '42daySumMaxTemp':['mean'], '42daySumHumidity':['mean'], '42daySumPrecip':['mean'],
                                                     '49daySumMaxTemp':['mean'], '49daySumHumidity':['mean'], '49daySumPrecip':['mean'],
                                                     '56daySumMaxTemp':['mean'], '56daySumHumidity':['mean'], '56daySumPrecip':['mean']})

In [ ]:
print(dfFWEW.count())
print(dfFWEW.describe())
print(dfFWEW.isnull().sum().sum())

In [ ]:
dfEval = dfFWEW

In [ ]:
# randomly select 6 years from 2010-2019 for training
dfTrain = dfEval[dfEval['YEAR'].isin([2010, 2011, 2012, 2013, 2014, 2015, 2016])]
dfValidate = dfEval[dfEval['YEAR'].isin([2017, 2018])]
dfTest = dfEval[dfEval['YEAR'].isin([2019, 2020])]

In [ ]:
# Store our random selection, run once
randomTrain = "RandomTrain"
dfTrain.to_sql(randomTrain, db_push_con, if_exists='replace', index=False)

randomTest = "RandomTest"
dfTest.to_sql(randomTest, db_push_con, if_exists='replace', index=False)

randomValidate = "RandomValidate"
dfValidate.to_sql(randomValidate, db_push_con, if_exists='replace', index=False)

In [ ]:
trainStats = tfdv.generate_statistics_from_dataframe(dfTrain)

In [ ]:
tfdv.visualize_statistics(trainStats)


In [ ]:
schema = tfdv.infer_schema(statistics=trainStats)
tfdv.display_schema(schema=schema)


In [ ]:
from sklearn.linear_model import RidgeCV
from sklearn.pipeline import make_pipeline
from sklearn.compose import TransformedTargetRegressor
from sklearn import svm
from sklearn.inspection import DecisionBoundaryDisplay


Y_train = dfTrain['SIZE_HA'].copy(deep=True)
Y_train.fillna(Y_train.mean(), inplace=True)

Y_train_discrete = dfTrain['size_ha_bin'].copy(deep=True)
Y_train_discrete.fillna(Y_train_discrete.min(), inplace=True)



X_train = dfTrain.drop(columns=['MONTH', 'SIZE_HA', 'OneMonth', 'OneYear', 'TwoMonth', 'TwoYear', 'EntryID', 'size_ha_bin', 'YEAR', 'DAY', 'FIRE_ID', 'FIRENAME', 'ClimateID', 'REP_DATE'])
X_train.fillna(X_train.mean(), inplace=True)
X_train_one = X_train.drop(columns=['TwoMeanTemp', 'TwoMinTemp', 'TwoMaxTemp', 'TwoMeanDewPoint', 'TwoMinDewPoint', 'TwoMaxDewPoint',
                                     'TwoMeanHumidity', 'TwoMinHumidity', 'TwoMaxHumidity', 'TwoMeanPressure', 'TwoMinPressure',
                                     'TwoMaxPressure', 'TwoMeanWindSpeed', 'TwoMinWindSpeed', 'TwoMaxWindSpeed', 'TwoMeanWindChill',
                                     'TwoMinWindChill', 'TwoMaxWindChill', 'TwoTotalPrecip', 'TwoMeanWindDirection'])
X_train_two = X_train.drop(columns=['OneMeanTemp', 'OneMinTemp', 'OneMaxTemp', 'OneMeanDewPoint', 'OneMinDewPoint', 'OneMaxDewPoint',
                                     'OneMeanHumidity', 'OneMinHumidity', 'OneMaxHumidity', 'OneMeanPressure', 'OneMinPressure',
                                     'OneMaxPressure', 'OneMeanWindSpeed', 'OneMinWindSpeed', 'OneMaxWindSpeed', 'OneMeanWindChill',
                                      'OneMinWindChill', 'OneMaxWindChill', 'OneTotalPrecip', 'OneMeanWindDirection'])

X_train_means_one = X_train_one.drop(columns=['OneMinTemp', 'OneMaxTemp', 'OneMinDewPoint', 'OneMaxDewPoint', 'OneMinHumidity', 'OneMaxHumidity',
                                              'OneMinPressure', 'OneMaxPressure', 'OneMinWindSpeed', 'OneMaxWindSpeed', 'OneMinWindChill',
                                              'OneMaxWindChill'])

X_train_means_two = X_train_two.drop(columns=['TwoMinTemp', 'TwoMaxTemp', 'TwoMinDewPoint', 'TwoMaxDewPoint', 'TwoMinHumidity', 'TwoMaxHumidity',
                                                'TwoMinPressure', 'TwoMaxPressure', 'TwoMinWindSpeed', 'TwoMaxWindSpeed', 'TwoMinWindChill',
                                                'TwoMaxWindChill'])

dfTrainScaled = dfTrain.copy(deep=True)
dfTrainScaled.fillna(dfTrainScaled.mean(), inplace=True)
dfTrainScaled = dfTrainScaled.drop(columns=['MONTH', 'OneMonth', 'OneYear', 'TwoMonth', 'TwoYear', 'EntryID', 'size_ha_bin', 'YEAR', 'DAY', 'FIRE_ID', 'FIRENAME', 'ClimateID', 'REP_DATE'])

# regularize y using log scale
Y_train = np.log(Y_train)
# regularize y values using z score
Y_train = (Y_train - Y_train.mean()) / Y_train.std()
# set max value to 3 zscore
Y_train[Y_train > 3] = 3

In [ ]:
dfTrainScaled = dfTrain.copy(deep=True)

In [ ]:
dfTrainScaled['SIZE_HA'] = np.log(dfTrainScaled['SIZE_HA'])
# regularize y values using z score
dfTrainScaled = (dfTrainScaled - dfTrainScaled.mean()) / dfTrainScaled.std()
# set max value to 3 zscore
dfTrainScaled[dfTrainScaled > 3] = 3
# set min value to -3 zscore
dfTrainScaled[dfTrainScaled < -3] = -3

# shift the wole train set to be positive
dfTrainScaled = dfTrainScaled + 3

In [ ]:
def scaleDF(df):
    dfScaled = df.copy(deep=True)
    dfScaled = (dfScaled - dfScaled.mean()) / dfScaled.std()
    dfScaled[dfScaled > 3] = 3
    dfScaled[dfScaled < -3] = -3
    dfScaled = dfScaled + 3
    return dfScaled

In [ ]:
trainStats2 = tfdv.generate_statistics_from_dataframe(dfTrainScaled)
tfdv.visualize_statistics(trainStats2)

In [ ]:
# one week data
weekData = pd.DataFrame()
train_dataset = dfTrainScaled.copy(deep=True)

# create a dataframe with the 7 day data
weekData['7daySumMaxTemp'] = train_dataset['7daySumMaxTemp']
weekData['7daySumHumidity'] = train_dataset['7daySumHumidity']
weekData['7daySumPrecip'] = train_dataset['7daySumPrecip']
weekData['SIZE_HA'] = train_dataset['SIZE_HA']


pairplotOne = sns.pairplot(weekData, kind="reg", diag_kind="kde")

In [ ]:
# one month data
monthData = pd.DataFrame()
train_dataset = dfTrainScaled.copy(deep=True)

# create a dataframe with the 28 day data
monthData['28daySumMaxTemp'] = train_dataset['28daySumMaxTemp']
monthData['28daySumHumidity'] = train_dataset['28daySumHumidity']
monthData['28daySumPrecip'] = train_dataset['28daySumPrecip']
monthData['SIZE_HA'] = train_dataset['SIZE_HA']

pairplotTwo = sns.pairplot(monthData, kind="reg", diag_kind="kde")

In [ ]:
# two month data
twoMonthData = pd.DataFrame()
train_dataset = dfTrainScaled.copy(deep=True)

# create a dataframe with the 56 day data
twoMonthData['56daySumMaxTemp'] = train_dataset['56daySumMaxTemp']
twoMonthData['56daySumHumidity'] = train_dataset['56daySumHumidity']
twoMonthData['56daySumPrecip'] = train_dataset['56daySumPrecip']
twoMonthData['SIZE_HA'] = train_dataset['SIZE_HA']

pairplotThree = sns.pairplot(twoMonthData, kind="reg", diag_kind="kde")

In [ ]:
dfTemp = pd.DataFrame()
dfTemp['SIZE_HA'] = dfTrainScaled['SIZE_HA'].copy(deep=True)
# categorize size_ha into 4 classes by quantile
dfTrainScaled['SIZE_BIN'] = pd.qcut(dfTemp['SIZE_HA'], 4, labels=False)

clf = svm.SVC(decision_function_shape='ovo')
clf.fit(dfTrainScaled.drop(columns=['SIZE_HA']), dfTrainScaled['SIZE_BIN'])

In [ ]:

print(dfTrainScaled.count())
print(dfTrainScaled.isna().sum().sum())
print(dfTrainScaled.dtypes)

## Define functions that build and train a model

The following code defines two functions:

  * `build_model(my_learning_rate)`, which builds a randomly-initialized model.
  * `train_model(model, feature, label, epochs)`, which trains the model from the examples (feature and label) you pass. 

Since you don't need to understand model building code right now, we've hidden this code cell.  You may optionally double-click the following headline to see the code that builds and trains a model.

In [ ]:
#@title Define the functions that build and train a model
def build_model(my_learning_rate):
  """Create and compile a simple linear regression model."""
  # Most simple tf.keras models are sequential.
  model = tf.keras.models.Sequential()

  # Describe the topography of the model.
  # The topography of a simple linear regression model
  # is a single node in a single layer.
  model.add(tf.keras.layers.Dense(units=1, 
                                  input_shape=(1,)))

  # Compile the model topography into code that TensorFlow can efficiently
  # execute. Configure training to minimize the model's mean squared error. 
  model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=my_learning_rate),
                loss="mean_squared_error",
                metrics=[tf.keras.metrics.RootMeanSquaredError()])

  return model        

def train_model(model, df, feature, label, epochs, batch_size):
  """Train the model by feeding it data."""

  # Feed the model the feature and the label.
  # The model will train for the specified number of epochs. 
  history = model.fit(x=df[feature],
                      y=df[label],
                      batch_size=batch_size,
                      epochs=epochs)

  # Gather the trained model's weight and bias.
  trained_weight = model.get_weights()[0]
  trained_bias = model.get_weights()[1]

  # The list of epochs is stored separately from the rest of history.
  epochs = history.epoch
  
  # Isolate the error for each epoch.
  hist = pd.DataFrame(history.history)

  # To track the progression of training, we're going to take a snapshot
  # of the model's root mean squared error at each epoch. 
  rmse = hist["root_mean_squared_error"]

  return trained_weight, trained_bias, epochs, rmse

def build_model_multi(my_learning_rate):
  """Create and compile a simple linear regression model."""
  # Most simple tf.keras models are sequential.
  model = tf.keras.models.Sequential()

  # Describe the topography of the model.
  # The topography of a simple linear regression model
  # is a single node in a single layer.
  model.add(tf.keras.layers.Dense(units=1, 
                                  input_shape=(3,)))

  # Compile the model topography into code that TensorFlow can efficiently
  # execute. Configure training to minimize the model's mean squared error. 
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=my_learning_rate),
                loss="mean_squared_error",
                metrics=[tf.keras.metrics.RootMeanSquaredError()])

  return model

def train_model_multi(model, df, dfFeatures, label, epochs, batch_size):
  """Train the model by feeding it data."""

  # Feed the model the feature and the label.
  # The model will train for the specified number of epochs. 
  history = model.fit(dfFeatures,
                      y=df[label],
                      batch_size=batch_size,
                      epochs=epochs)

  # Gather the trained model's weight and bias.
  trained_weight = model.get_weights()[0][0]
  trained_bias = model.get_weights()[1][0]

  # The list of epochs is stored separately from the rest of history.
  epochs = history.epoch
  
  # Isolate the error for each epoch.
  hist = pd.DataFrame(history.history)

  # To track the progression of training, we're going to take a snapshot
  # of the model's root mean squared error at each epoch. 
  rmse = hist["root_mean_squared_error"]

  return trained_weight, trained_bias, epochs, rmse

print("Defined the build_model and train_model functions.")

## Define plotting functions

The following [matplotlib](https://developers.google.com/machine-learning/glossary/#matplotlib) functions create the following plots:

*  a scatter plot of the feature vs. the label, and a line showing the output of the trained model
*  a loss curve

You may optionally double-click the headline to see the matplotlib code, but note that writing matplotlib code is not an important part of learning ML programming.

In [ ]:
#@title Define the plotting functions
def plot_the_model(trained_weight, trained_bias, feature, label):
  """Plot the trained model against 200 random training examples."""

  # Label the axes.
  plt.xlabel(feature)
  plt.ylabel(label)

  # Create a scatter plot from 200 random points of the dataset.
  random_examples = dfTrainScaled.sample(n=dfTrainScaled.shape[0])
  plt.scatter(random_examples[feature], random_examples[label])

  # Create a red line representing the model. The red line starts
  # at coordinates (x0, y0) and ends at coordinates (x1, y1).
  x0 = 0
  y0 = trained_bias
  x1 = 6
  y1 = trained_bias + (trained_weight * x1)
  plt.plot([x0, x1], [y0, y1], c='r')

  # Render the scatter plot and the red line.
  plt.show()


def plot_the_loss_curve(epochs, rmse):
  """Plot a curve of loss vs. epoch."""

  plt.figure()
  plt.xlabel("Epoch")
  plt.ylabel("Root Mean Squared Error")

  plt.plot(epochs, rmse, label="Loss")
  plt.legend()
  plt.ylim([rmse.min()*0.97, rmse.max()])
  plt.show()  

print("Defined the plot_the_model and plot_the_loss_curve functions.")

## Call the model functions

An important part of machine learning is determining which [features](https://developers.google.com/machine-learning/glossary/#feature) correlate with the [label](https://developers.google.com/machine-learning/glossary/#label). For example, real-life home-value prediction models typically rely on hundreds of features and synthetic features. However, this model relies on only one feature. For now, you'll arbitrarily use `total_rooms` as that feature. 


In [ ]:
# The following variables are the hyperparameters.
learning_rate = 0.02
epochs = 30
batch_size = 3

# Specify the feature and the label.
single_feature = "28daySumMaxTemp"  # the total number of rooms on a specific city block.
single_label="SIZE_HA" # the median value of a house on a specific city block.
#my_label="size_ha_bin"
# That is, you're going to create a model that predicts house value based 
# solely on total_rooms.  

# Discard any pre-existing version of the model.
single_model = None

# Invoke the functions.
single_model = build_model(learning_rate)
weight, bias, epochs, rmse = train_model(single_model, dfTrainScaled, 
                                         single_feature, single_label,
                                         epochs, batch_size)

print("\nThe learned weight for your model is %.4f" % weight)
print("The learned bias for your model is %.4f\n" % bias )

plot_the_model(weight, bias, single_feature, single_label)
plot_the_loss_curve(epochs, rmse)

## Use the model to make predictions

You can use the trained model to make predictions. In practice, [you should make predictions on examples that are not used in training](https://developers.google.com/machine-learning/crash-course/training-and-test-sets/splitting-data). However, for this exercise, you'll just work with a subset of the same training dataset. A later Colab exercise will explore ways to make predictions on examples not used in training.

First, run the following code to define the house prediction function:

In [ ]:
def predict_values(predictionmodel, n, df, feature, label):
  """Predict values based on a feature."""

  scaled = scaleDF(df)
  batch = scaled.iloc[0:n,df.columns.get_loc(feature)]
  predicted_values = predictionmodel.predict_on_batch(x=batch)

  print("feature   label          predicted    ")
  print("  value   value          value        ")
  print("                                      ")
  print("--------------------------------------")

  for index, row in batch.iterrows():
    print("%5.0f %6.0f %15.0f" % (row[feature], scaled[index][label], predicted_values[index][0]))

In [ ]:

predict_values(single_model, 10, dfTrainScaled, single_feature, single_label)

In [ ]:
single_model.evaluate(x=scaleDF(dfTest)[single_feature], y=scaleDF(dfTest)[single_label], batch_size=2)

In [ ]:
# The following variables are the hyperparameters.
learning_rate = 0.02
epochs = 30
batch_size = 3

# Specify the feature and the label.
my_feature = dfTrainScaled[['28daySumMaxTemp','28daySumHumidity', '28daySumPrecip']] 
# create numpy stack of features
my_feature_stack = np.stack(( dfTrainScaled[['28daySumMaxTemp']], 
                                dfTrainScaled[['28daySumHumidity']], 
                                dfTrainScaled[['28daySumPrecip']]), axis=1)

my_label="SIZE_HA" # the median value of a house on a specific city block.
#my_label="size_ha_bin"
# That is, you're going to create a model that predicts house value based 
# solely on total_rooms.  

# Discard any pre-existing version of the model.
my_model = None

# Invoke the functions.
my_model = build_model_multi(learning_rate)
weight, bias, epochs, rmse = train_model_multi(my_model, dfTrainScaled, 
                                         my_feature_stack, my_label,
                                         epochs, batch_size)

#print("\nThe learned weight for your model is %.4f" % weight)
#print("The learned bias for your model is %.4f\n" % bias )

#plot_the_model(weight, bias, my_feature, my_label)
#plot_the_loss_curve(epochs, rmse)

In [ ]:
# evaluate the model prediction accuracy

multi_stack = np.stack(( dfTest[['28daySumMaxTemp']], dfTest[['28daySumHumidity']], dfTest[['28daySumPrecip']]), axis=1)
my_model.evaluate(x=multi_stack, y=dfTest[my_label], batch_size=1, verbose=1)